In [ ]:
!pip install openbb

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [29]:
import pandas as pd
import requests
import os
import numpy as np, pandas as pd
from datetime import datetime, timezone
from math import sqrt, exp, log
from scipy.stats import norm
from scipy.optimize import brentq
from scipy.interpolate import griddata
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401
import yfinance as yf
from openbb import obb

In [53]:
TICKER='QQQ'

In [54]:
get_chains = obb.derivatives.options.chains(symbol=TICKER)

In [55]:
ALL = get_chains.to_dataframe()
ALL.rename(columns={'expiration': 'expiry'}, inplace=True)
ALL.rename(columns={'option_type':'type'}, inplace = True)

In [56]:
ASOF = datetime.now(timezone.utc).date()
ALL['asof']=pd.to_datetime(ASOF)

In [57]:
display(ALL.head(10))

,underlying_symbol,underlying_price,contract_symbol,expiry,dte,strike,type,open_interest,volume,last_trade_price,last_trade_time,bid,ask,change,change_percent,implied_volatility,in_the_money,currency,asof
0,QQQ,622.92,QQQ251204P00500000,2025-12-04,0,500.0,put,0,2,0.01,2025-12-03 12:25:16-05:00,0.0,0.0,0.0,0.0,0.500005,False,USD,2025-12-04
1,QQQ,622.92,QQQ251204P00505000,2025-12-04,0,505.0,put,0,1,0.03,2025-11-28 12:26:18-05:00,0.0,0.0,0.0,0.0,0.500005,False,USD,2025-12-04
2,QQQ,622.92,QQQ251204P00510000,2025-12-04,0,510.0,put,0,65,0.01,2025-12-02 13:35:49-05:00,0.0,0.0,0.0,0.0,0.500005,False,USD,2025-12-04
3,QQQ,622.92,QQQ251204P00515000,2025-12-04,0,515.0,put,0,20,0.02,2025-12-02 14:04:20-05:00,0.0,0.0,0.0,0.0,0.500005,False,USD,2025-12-04
4,QQQ,622.92,QQQ251204P00520000,2025-12-04,0,520.0,put,0,77,0.02,2025-12-01 15:33:37-05:00,0.0,0.0,0.0,0.0,0.500005,False,USD,2025-12-04
5,QQQ,622.92,QQQ251204P00525000,2025-12-04,0,525.0,put,0,3,0.01,2025-12-03 09:31:58-05:00,0.0,0.0,0.0,0.0,0.500005,False,USD,2025-12-04
6,QQQ,622.92,QQQ251204P00530000,2025-12-04,0,530.0,put,0,17,0.01,2025-12-03 14:27:59-05:00,0.0,0.0,0.0,0.0,0.500005,False,USD,2025-12-04
7,QQQ,622.92,QQQ251204P00535000,2025-12-04,0,535.0,put,0,817,0.01,2025-12-03 09:58:39-05:00,0.0,0.0,0.0,0.0,0.500005,False,USD,2025-12-04
8,QQQ,622.92,QQQ251204C00540000,2025-12-04,0,540.0,call,0,61,83.82,2025-12-03 15:08:58-05:00,0.0,0.0,0.0,0.0,0.000010,True,USD,2025-12-04
9,QQQ,622.92,QQQ251204P00540000,2025-12-04,0,540.0,put,0,101,0.01,2025-12-03 15:22:06-05:00,0.0,0.0,0.0,0.0,0.500005,False,USD,2025-12-04


As most of the data is untradable, moving forward we will only use rows with bid, ask and volume > 0

In [69]:
ALL = ALL[(ALL["bid"] > 0) & (ALL["ask"] > 0) & (ALL["volume"] > 0)]

In [70]:
try:
    ALL.to_parquet('/content/drive/MyDrive/Colab Notebooks/VolSurf_ML/combined_options_data.parquet', index=False)
    print("Successfully saved combined_options_data.parquet to Google Drive.")
except Exception as e:
    print(f"Error saving file to Google Drive: {e}")
    print("Make sure you have mounted your Google Drive correctly.")

Successfully saved combined_options_data.parquet to Google Drive.


In [71]:
# Underlying historical prices (for realised vol, returns, etc.)
hist_price = yf.Ticker(TICKER).history(period="2y")  # choose horizon
hist_price_path = '/content/drive/MyDrive/Colab Notebooks/VolSurf_ML/hist_price.parquet'
hist_price.to_parquet(hist_price_path)
print(f"Saved underlying history to {hist_price_path}")


Saved underlying history to /content/drive/MyDrive/Colab Notebooks/VolSurf_ML/hist_price.parquet


In [68]:
print("Total rows:", len(ALL))

print("\nBid/ask quality:")
print("  both bid>0, ask>0:", ((ALL["bid"] > 0) & (ALL["ask"] > 0)).sum())
print("  only bid>0:", ((ALL["bid"] > 0) & (ALL["ask"] == 0)).sum())
print("  only ask>0:", ((ALL["bid"] == 0) & (ALL["ask"] > 0)).sum())
print("  both 0:", ((ALL["bid"] == 0) & (ALL["ask"] == 0)).sum())

print("\nVolume / OI:")
print("  volume>0:", (ALL["volume"] > 0).sum())
print("  open_interest>0:", (ALL["open_interest"] > 0).sum())

print("\nImplied vol nonzero:", (ALL["implied_volatility"] > 0).sum())

tradable = ALL[(ALL["bid"] > 0) & (ALL["ask"] > 0)]
print("Tradable rows:", len(tradable))
print(tradable.groupby("dte").size().head(20))   # or groupby("expiry") if you’ve converted

Total rows: 7435

Bid/ask quality:
  both bid>0, ask>0: 445
  only bid>0: 0
  only ask>0: 2
  both 0: 6988

Volume / OI:
  volume>0: 7192
  open_interest>0: 580

Implied vol nonzero: 7435
Tradable rows: 445
dte
15     38
27     17
43     52
57      1
78     14
106    21
117    16
134    14
162     5
196    66
208    19
288    32
300     5
379    33
407    27
560    28
652     9
743    29
778    19
dtype: int64
